In [1]:
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Grid, Bar
import pandas as pd
import akshare as ak
import pickle
from utils.log import logger
from utils.fetch_stock_data import get_data_from_all
# 读取temp_data/stock_data_20210701_20230328.pkl
with open('temp_data/stock_data_20210701_20230328.pkl', 'rb') as f:
    stock_data = pickle.load(f)

df = stock_data[('600519', '贵州茅台')]
type(df['vol'].values.tolist()[0])

float

In [4]:


# 读取股票数据
df = stock_data[('600519', '贵州茅台')]


def kline_compare_base(stock_df,axis_index = 0):
    # 将数据转换为列表格式
    data = stock_df[['open', 'close', 'low', 'high']].values.tolist()
    dates = stock_df.index.tolist() # date type:str, like '2021-07-01'

    # 绘制 K 线图
    kline = (
        Kline(init_opts=opts.InitOpts(bg_color="#000000"))

        

        .add_xaxis(dates)
        .add_yaxis(
            series_name = "K线图",
            y_axis = data,
            xaxis_index=axis_index,
            yaxis_index=axis_index,
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                color0="#00da3c",
                border_color="#8A0000",
                border_color0="#008F28",
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="双品种组合"),
            xaxis_opts=opts.AxisOpts(
                # type_='category',
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            legend_opts=opts.LegendOpts(
                is_show=True, textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))
        )
        .set_series_opts(
            # label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
            label_opts=opts.LabelOpts(is_show=False)
        )
    )

    # 绘制均线图
    # 计算均线数据
    ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    ma20 = stock_df['close'].rolling(window=20).mean().tolist()


    line = (
        Line()
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
        )
        .add_xaxis(dates)
        .add_yaxis("MA5", ma5, symbol='none')  # symbol='none'去掉line上的圆点
        .add_yaxis("MA10", ma10, symbol='none')
        .add_yaxis("MA20", ma20, symbol='none')
    )

    # 将 K 线图和均线图组合在一起
    k_ma  = kline.overlap(line)
    return  k_ma

def stock_compare_vol(stock_df,axis_index = 0):
    data = df['vol'].astype(int).values.tolist()
    # data = stock_df[['vol']].values.tolist()

    print(data)
    dates = stock_df.index.tolist() # date type:str, like '2021-07-01'
    bar = (
        Bar()

        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                # type_='category',
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            legend_opts=opts.LegendOpts(
                is_show=False, textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))
        )

        .add_xaxis(dates)
        .add_yaxis(
            series_name = "成交量",
            y_axis = data,
            xaxis_index=axis_index,
            yaxis_index=axis_index,
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                # color0="#00da3c",
                # border_color="#8A0000",
                # border_color0="#008F28",
            ),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False)
        )
    )

    # 绘制均线图
    # 计算均线数据
    # ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    # ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    # ma20 = stock_df['close'].rolling(window=20).mean().tolist()


    # line = (
    #     Line()
    #     .set_series_opts(
    #         label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
    #     )
    #     .add_xaxis(dates)
    #     .add_yaxis("MA5", ma5, symbol='none')  # symbol='none'去掉line上的圆点
    #     .add_yaxis("MA10", ma10, symbol='none')
    #     .add_yaxis("MA20", ma20, symbol='none')
    # )

    # 将 K 线图和均线图组合在一起
    # k_ma  = bar.overlap(line)
    return  bar


In [8]:
def kline_compare(df_top,df_bottom,start_value,end_value):
    
    kline_top = kline_compare_base(df_top,axis_index=0)
    kline_bottom = kline_compare_base(df_bottom,axis_index=1)
    kline_top_vol = stock_compare_vol(df_top,axis_index=2)

    kline_top.set_global_opts(datazoom_opts=[
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,0], pos_top="94%",pos_bottom="4%"), 
        opts.DataZoomOpts(is_show=True,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,1], pos_top="97%", pos_bottom="1%"),
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,2], pos_top="97%", pos_bottom="1%"),
        ],
        # xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False)),
    )
    
    grid = Grid(init_opts=opts.InitOpts(width="1600px", height="800px"))
    
    grid.add(kline_top, grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))
    grid.add(kline_bottom, grid_opts=opts.GridOpts(pos_top="60%", pos_bottom="0%"))
    grid.add(kline_top_vol, grid_opts=opts.GridOpts(pos_top="40%", pos_bottom="40%"))

    return grid


# df_top=stock_data[('600519', '贵州茅台')]
df_top=get_data_from_all('600519',stock_data)
df_bottom = stock_data[('600172', '黄河旋风')]
kline_compare(df_top,df_bottom,'2021-08-06','2022-03-29').render_notebook()

[33701, 46275, 34421, 35283, 28122, 35517, 30908, 39010, 29925, 30771, 42190, 35408, 30052, 23475, 29780, 39090, 47585, 98619, 86577, 85369, 63864, 77736, 97401, 54375, 38175, 48060, 45686, 47698, 83285, 38663, 47551, 37905, 28057, 61201, 59970, 41292, 114545, 71597, 72594, 54239, 54020, 36095, 51588, 43677, 77316, 42837, 52835, 55338, 31211, 33064, 19950, 33388, 27720, 33761, 41570, 45968, 55130, 31743, 30342, 69537, 126869, 49221, 43291, 40539, 47452, 37676, 30606, 50143, 26623, 28561, 69652, 40315, 28663, 23019, 35963, 32461, 21130, 34647, 29491, 28776, 41690, 32500, 27829, 31130, 27077, 17706, 27352, 35349, 22652, 17607, 24947, 33526, 24505, 19521, 31156, 42573, 45782, 54512, 39753, 32656, 42399, 36925, 26254, 20422, 35818, 31064, 36980, 61048, 64239, 43148, 65807, 40449, 30078, 37211, 48403, 32258, 26286, 17431, 39099, 56130, 37556, 27651, 54049, 35261, 29665, 33843, 28396, 51795, 29817, 29627, 22277, 27474, 56747, 29322, 27354, 31216, 26371, 34014, 30813, 23986, 25510, 23905, 426